# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846814066043                   -0.70    4.8         
  2   -7.852322221402       -2.26       -1.53    1.0   16.7ms
  3   -7.852617244679       -3.53       -2.56    1.5   18.5ms
  4   -7.852646045678       -4.54       -2.90    2.5   22.3ms
  5   -7.852646524674       -6.32       -3.20    1.0   17.0ms
  6   -7.852646680990       -6.81       -4.25    1.0   17.0ms
  7   -7.852646686684       -8.24       -5.01    2.2   22.4ms
  8   -7.852646686726      -10.38       -5.51    1.5   19.3ms
  9   -7.852646686728      -11.58       -5.66    1.2   17.9ms
 10   -7.852646686730      -11.87       -6.71    1.0   17.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846773261210                   -0.70           4.5         
  2   -7.852552750822       -2.24       -1.64   0.80    2.2    216ms
  3   -7.852638554617       -4.07       -2.74   0.80    1.0   61.8ms
  4   -7.852646515518       -5.10       -3.32   0.80    2.0   19.3ms
  5   -7.852646677026       -6.79       -4.23   0.80    1.8   18.0ms
  6   -7.852646686595       -8.02       -4.82   0.80    2.0   19.7ms
  7   -7.852646686721       -9.90       -5.70   0.80    1.5   17.1ms
  8   -7.852646686730      -11.06       -6.85   0.80    2.0   19.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.380094e+01     3.291980e+00
 * time: 0.04595613479614258
     1     1.232801e+00     2.033710e+00
 * time: 0.27105712890625
     2    -1.254125e+00     2.517570e+00
 * time: 0.2887611389160156
     3    -3.691819e+00     2.035582e+00
 * time: 0.3141341209411621
     4    -4.701090e+00     1.870208e+00
 * time: 0.3395850658416748
     5    -6.644361e+00     1.055905e+00
 * time: 0.3649439811706543
     6    -7.371995e+00     4.719368e-01
 * time: 0.3902881145477295
     7    -7.668230e+00     2.394569e-01
 * time: 0.408005952835083
     8    -7.745229e+00     9.681594e-02
 * time: 0.42566895484924316
     9    -7.763456e+00     1.153055e-01
 * time: 0.44341301918029785
    10    -7.775555e+00     4.979200e-02
 * time: 0.4610869884490967
    11    -7.780884e+00     4.332608e-02
 * time: 0.47877001762390137
    12    -7.786047e+00     5.110680e-02
 * time: 0.4964289665222168
    13    -7.794195e+00     6.378840e-02
 * time: 0.514096975

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846801867260                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645865896                   -1.64         
  2   -7.852646686730       -6.09       -3.70    1.55s
  3   -7.852646686730      -13.22       -7.22    125ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.5105794429729203e-7
|ρ_newton - ρ_scfv| = 6.19616350579379e-8
|ρ_newton - ρ_dm|   = 4.267566857927198e-9
